In [ ]:
# Импорт библиотек
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import easyocr

In [ ]:
# проверка пар, сбор списка пар

rootdir = "./datasets/405/"
regex_find = re.compile('(.*csv$)|(.*jpg$)')
jpg_list = []
csv_list = []

for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex_find.match(file):
            temp = os.path.splitext(file)
            if temp[1] == '.jpg':
                name = temp[0] + temp[1]
                clean_name = temp[0].strip() + temp[1]
                if name != clean_name:
                    os.rename(rootdir+'/'+name, rootdir+'/'+clean_name) # убираем лишние пробелы в названиях фото
                jpg_list.append(temp[0])
            else: csv_list.append(temp[0])

for x in jpg_list:
    if x not in csv_list:
        print(f'Отсутствует csv для {x}.jpg')

for x in csv_list:
    if x not in jpg_list:
        print(f'Отсутствует jpg для {x}.csv')

pairs_list = list(set(jpg_list) & set(csv_list))

In [ ]:
# инициализация ридера easyocr
reader = easyocr.Reader(['ru'])

# переменные для рассчёта точности
sum_acc = 0
iteration = 0

# перебор всех изображений с парой
for x in pairs_list:
    #открыть изображение и предсказать ридером
    image_path = './datasets/405/'+x+'.jpg'
    image = cv2.imread(image_path)
    result = reader.readtext(image)
    
    # вытащить из результата ocr даты, виды и типы донаций
    date_r = re.compile('^(0[1-9]|[12][0-9]|3[01])[- /.|,| |-](0[1-9]|1[012])[- /.|,| |-](19|20)\d\d$')
    dates = []
    bloodstypes = []
    volumes = []
    index = 0
    
    for i in range(len(result)):
        try:
            temp = result[index]
            text = temp[1]
            if date_r.match(text):
                text = text.replace(',', '.')
                text = text.replace('-', '.')
                text = text.replace(' ', '.')
                dates.append(text)
                temp = result[index+1]
                text = temp[1]
                text = text.strip(' ')
                bloodstypes.append(text)
                temp = result[index+2]
                text = temp[1]
                volumes.append(text)
                index += 3
            else:
                index += 1
        except: continue
            
    # разбить bloodstypes на списки видов донации и типов донации
    pattern = r'\((.*?)\)'
    bloods = []
    types = []

    for row in bloodstypes:
        matches = re.findall(pattern, row)
        t = ' '.join(matches)
        types.append(t)
        b = re.sub(pattern, '', row)
        b = b.strip(' ')
        bloods.append(b)
        
    # переименовать виды и типы донации
    for i in range(len(bloods)):
        try:
            if (bloods[i][0] in ['к', 'р']) | (bloods[i][1] == 'р'):
                bloods[i] = 'Цельная кровь'
            elif (bloods[i][0] in ['п','л']) | (bloods[i][1] == 'л'):
                bloods[i] = 'Плазма'
            elif bloods[i][0] == 'ц':
                bloods[i] = 'Тромбоциты'
            else: continue
        except: continue

    for i in range(len(types)):
        try:
            if (types[i][0] == 'б') | (types[i][1] == 'в'):
                types[i] = 'Безвозмездно'
            elif (types[i][0] in ['п','л','а','т']) | (types[i][1] == ['п','л','а','т']):
                types[i] = 'Платно'
            else: continue
        except: continue
    
    # собрать датафрейм и сортировать по дате донации
    df_pred = pd.DataFrame(
    columns=[
        'Класс крови', 
        'Дата донации', 
        'Тип донации'
    ]
    )

    for i in range(len(dates)):

        df_pred.loc[i, 'Класс крови'] = bloods[i]
        df_pred.loc[i, 'Дата донации'] = dates[i]
        df_pred.loc[i, 'Тип донации'] = types[i]

    df_pred['Дата донации'] = pd.to_datetime(df_pred['Дата донации'], dayfirst=True)
    df_pred.sort_values(by=['Дата донации'], inplace=True, ignore_index=True)
    df_pred['Дата донации'] = df_pred['Дата донации'].astype('str')
    df_pred.dropna(inplace=True)
    
    # открыть таблицу для сравнения
    df_true = pd.read_csv('./datasets/405/'+x+'.csv').drop(
    [
        'ID', 
        'ID пользователя', 
        'Дата добавления донации', 
        'Регион', 'Место стадчи', 
        'Статус донации',
        'Есть справка'
    ], axis=1
    )
    
    # преобразовать в datetime и обратно в строку, чтобы форматирование совпадало
    df_true['Дата донации']= pd.to_datetime(df_true['Дата донации'], dayfirst=True)
    df_true['Дата донации'] = df_true['Дата донации'].astype('str')
    
    
    # рассчёт точности для картинки
    true_counter = 0
    counter = 0
    
    for row_true in range(len(df_true)):
        for row_pred in range(len(df_pred)):
            if df_true.loc[row_true, 'Дата донации'] == df_pred.loc[row_pred, 'Дата донации']:
                true_temp_counter = 1
                if df_true.loc[row_true, 'Класс крови'] == df_pred.loc[row_pred, 'Класс крови']:
                    true_temp_counter += 1
                if df_true.loc[row_true, 'Тип донации'] == df_pred.loc[row_pred, 'Тип донации']:
                    true_temp_counter += 1
                true_counter += true_temp_counter/3
        counter += 1
    sum_acc += true_counter/counter
    iteration += 1
    
#рассчёт общей точности
result = sum_acc/iteration
print(result)